Trained and tested except Frechet

In [ ]:
%load_ext autoreload
%autoreload 2

# Environment prep for Colab
 - download git repo 
 - install required pip packages

In [ ]:
!git clone https://github.com/kkahloots/Generative_Models.git # this is for loading git with correct brach
%cd /content/Generative_Models/
!pip -q install -r /content/Generative_Models/requirements.txt
%cd /content/

Cloning into 'Generative_Models'...
remote: Enumerating objects: 15891, done.
remote: Total 15891 (delta 0), reused 0 (delta 0), pack-reused 15891
Receiving objects: 100% (15891/15891), 733.46 MiB | 44.36 MiB/s, done.
Resolving deltas: 100% (10845/10845), done.
Checking out files: 100% (1697/1697), done.
/content/Generative_Models
/content


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!mkdir /content/drive/My\ Drive/Results
MAIN_SAVE_DIR = '/content/drive/My Drive/Results'
IMG_DIR = '/content/Generative_Models/data/.pokemon' #change .qbert here as well!git clone https://github.com/kkahloots/Generative_Models.git # this is for loading git with correct brach
%cd /content/Generative_Models/
!pip -q install -r /content/Generative_Models/requirements.txt
!pip install colorlog==4.0.2
%cd /content/

Mounted at /content/drive
mkdir: cannot create directory ‘/content/drive/My Drive/Results’: File exists
/content/Generative_Models
/content


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Connect Drive to save and restore results

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!mkdir /content/drive/My\ Drive/Results
MAIN_SAVE_DIR = '/content/drive/My Drive/Results'
IMG_DIR = '/content/Generative_Models/data/.pokemon' #change .qbert here as well

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mkdir: cannot create directory ‘/content/drive/My Drive/Results’: File exists


# Settings

In [ ]:
%env TF_KERAS = 1
import os
sep_local = os.path.sep

import sys
sys.path.append('..'+sep_local+'..')
print(sep_local)

env: TF_KERAS=1
/


In [ ]:
import tensorflow as tf
print(tf.__version__)

2.3.0


In [ ]:
#os.chdir('..'+sep_local+'..'+sep_local+'..'+sep_local+'..'+sep_local+'..')
os.chdir('/content/Generative_Models')
print(os.getcwd())

/content/Generative_Models


# only on colab

In [ ]:
!cat data/.pokemon/pokemon.tar.part* > data/.pokemon/pokemon.tar

In [ ]:
!tar -xvf data/.pokemon/pokemon.tar --directory data/.pokemon/

Pokemon_LMDB/_training/
Pokemon_LMDB/_training/lock.mdb
Pokemon_LMDB/_training/data.mdb
Pokemon_LMDB/_validation/
Pokemon_LMDB/_validation/lock.mdb
Pokemon_LMDB/_validation/data.mdb
Pokemon_LMDB/meta_info.json


# Dataset loading

In [ ]:
from training.generators.from_lmdb.lmdb_image_generator import get_generators

In [ ]:
dataset_name='pokemon'
inputs_shape= image_shape=(100, 100, 3)
batch_size = 10
latents_dim = 100
intermediate_dim = 50
lmdb_dir = 'data/.pokemon/Pokemon_LMDB'

In [ ]:
from transformation.lmdb_transformer import LmdbTransformer

In [ ]:
train_generator, val_generator = get_generators(
        lmdb_dir=lmdb_dir,
        batch_size=batch_size,
        episode_len=None,
        episode_shift=None
)

/content/Generative_Models/transformation/file_image_generator.py:51: UserWarning: No files found
  warnings.warn(msg)


Initializing Iterator training Number of images 591
Initializing Iterator validation Number of images 218


In [ ]:
import numpy as np
_instance_scale=1.0
for data in val_generator:
    print(np.max(data['images']))
    break
    #print(data.keys())

1.0


In [ ]:
data['images'][0].shape

TensorShape([100, 100, 3])

In [ ]:
data.keys()

dict_keys(['images', 'label'])

# Model's Layers definition

In [ ]:
import numpy as np
from collections.abc import Iterable
if isinstance(inputs_shape, Iterable):
    flat_outputs_shape = np.prod(inputs_shape)
flat_outputs_shape

30000

In [ ]:
latents_dim

100

In [ ]:
enc_lays = [tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(units=latents_dim)]

dec_lays = [tf.keras.layers.Dense(units=latents_dim, activation='relu'),
            tf.keras.layers.Dense(units= latents_dim*intermediate_dim//3 , activation='relu'),
            tf.keras.layers.Dense(units= latents_dim*intermediate_dim//3 , activation='relu'),
            tf.keras.layers.Dense(units= latents_dim*intermediate_dim//3 , activation='relu'),

            tf.keras.layers.Dense(units=flat_outputs_shape),
            tf.keras.layers.Reshape(inputs_shape)]

# Model definition

In [ ]:
model_name = dataset_name+'AE_Dense_reconst_1ell_01sharpdiff'
experiments_dir='/content/drive/My Drive/Results'+sep_local+model_name
print(experiments_dir)

/content/drive/My Drive/Results/pokemonAE_Dense_reconst_1ell_01sharpdiff


In [ ]:
from training.autoencoding_basic.autoencoders.autoencoder import autoencoder as AE

In [ ]:
variables_params = \
[
    {
        'name': 'inference', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latents_dim,
        'layers': enc_lays
    }

    ,
    
        {
        'name': 'generative', 
        'inputs_shape':latents_dim,
        'outputs_shape':inputs_shape,
        'layers':dec_lays
    }
]

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist

In [ ]:
_restore = os.path.join(experiments_dir, 'var_save_dir')

In [ ]:
create_if_not_exist(_restore)
_restore

'/content/drive/My Drive/Results/pokemonAE_Dense_reconst_1ell_01sharpdiff/var_save_dir'

In [ ]:
#to restore trained model, set filepath=_restore

In [ ]:
ae = AE( 
    name=model_name,
    latents_dim=latents_dim,
    batch_size=batch_size,
    variables_params=variables_params, 
    filepath=_restore
    )

  DEBUG    | Restore old models ...
  DEBUG    | Restore old models ...


  WARNING  | None
  WARNING  | None


Model: "inference"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_inputs (InputLayer [(None, 100, 100, 3)]     0         
_________________________________________________________________
dense (Dense)                (None, 100, 100, 50)      200       
_________________________________________________________________
dense_1 (Dense)              (None, 100, 100, 50)      2550      
_________________________________________________________________
flatten (Flatten)            (None, 500000)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               50000100  
_________________________________________________________________
batch_normalization (BatchNo (None, 100)               400       
_________________________________________________________________
dropout (Dropout)            (None, 100)               0 

  WARNING  | None
  WARNING  | None


Model: "generative"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
generative_inputs (InputLaye [(None, 100)]             0         
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 1666)              168266    
_________________________________________________________________
dense_5 (Dense)              (None, 1666)              2777222   
_________________________________________________________________
dense_6 (Dense)              (None, 1666)              2777222   
_________________________________________________________________
dense_7 (Dense)              (None, 30000)             50010000  
_________________________________________________________________
reshape (Reshape)            (None, 100, 100, 3)       0

In [ ]:
from evaluation.quantitive_metrics.sharp_difference import prepare_sharpdiff

#from statistical.losses_utilities import similarty_to_distance
from statistical.ae_losses import expected_loglikelihood_with_lower_bound as ellwlb

In [ ]:
#ae.compile(metrics=None)
#ae.compile()
ae.compile(loss={'x_logits': lambda x_true, x_logits: ellwlb(x_true, x_logits)+ 0.1*prepare_sharpdiff([ae.batch_size]+ae.get_inputs_shape())(x_true, x_logits)})

Model: "pokemonAE_Dense_reconst_1ell_01sharpdiff"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_inputs (InputLayer [(None, 100, 100, 3)]     0         
_________________________________________________________________
inference (Functional)       (None, 100)               50003250  
_________________________________________________________________
generative (Functional)      (None, 100, 100, 3)       55742822  
_________________________________________________________________
tf_op_layer_x_logits (Tensor [(None, 100, 100, 3)]     0         
Total params: 105,746,072
Trainable params: 105,745,866
Non-trainable params: 206
_________________________________________________________________
None


In [ ]:
ae.output_shape

(None, 100, 100, 3)

# Callbacks

In [ ]:
from training.callbacks.sample_generation import SampleGeneration
from training.callbacks.save_model import ModelSaver

In [ ]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    min_delta=1e-15, 
    patience=10, 
    verbose=1, 
    restore_best_weights=False
)

In [ ]:
ms = ModelSaver(filepath=_restore)

In [ ]:
csv_dir = os.path.join(experiments_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, ae.name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
csv_dir

'/content/drive/My Drive/Results/pokemonAE_Dense_reconst_1ell_01sharpdiff/csv_dir/pokemonAE_Dense_reconst_1ell_01sharpdiff.csv'

In [ ]:
image_gen_dir = os.path.join(experiments_dir, 'image_gen_dir')
create_if_not_exist(image_gen_dir)

In [ ]:
sg = SampleGeneration(latents_shape=latents_dim, filepath=image_gen_dir, gen_freq=5, save_img=True, gray_plot=False)

In [ ]:
import numpy as np

# Model Training

In [ ]:
ae.fit(
    x=train_generator,
    input_kw='images',
    steps_per_epoch=int(5e3),
    epochs=int(1e6), 
    verbose=2,
    callbacks=[es, ms, csv_log, sg],
    workers=-1,
    use_multiprocessing=True,
    validation_data=val_generator,
    validation_steps=int(1e4)
)

Epoch 1/1000000

Epoch 00001: loss improved from inf to 4000.88159, saving model to /content/drive/My Drive/Results/pokemonAE_Dense_reconst_1ell_01sharpdiff/var_save_dir
5000/5000 - 612s - loss: 4000.8816 - psnr: 13.2424 - total_variation: 102658.3672 - ssim_multiscale: 6.7675 - sharpdiff: 5.3225 - mean_absolute_error: 0.1214 - mean_squared_error: 0.0482 - val_loss: 6251.0254 - val_psnr: 12.1432 - val_total_variation: 22254.9453 - val_ssim_multiscale: 4.3933 - val_sharpdiff: 9.2693 - val_mean_absolute_error: 0.1865 - val_mean_squared_error: 0.0644
Epoch 2/1000000

Epoch 00002: loss improved from 4000.88159 to 3966.60840, saving model to /content/drive/My Drive/Results/pokemonAE_Dense_reconst_1ell_01sharpdiff/var_save_dir
5000/5000 - 615s - loss: 3966.6084 - psnr: 13.2726 - total_variation: 102759.2969 - ssim_multiscale: 6.7876 - sharpdiff: 5.3245 - mean_absolute_error: 0.1197 - mean_squared_error: 0.0478 - val_loss: 5849.5669 - val_psnr: 12.3519 - val_total_variation: 21044.7227 - val_

# Model Evaluation

## inception_score

In [ ]:
from evaluation.generativity_metrics.inception_metrics import inception_score

In [ ]:
is_mean, is_sigma = inception_score(ae, tolerance_threshold=1e-6, max_iteration=300)
print(f'inception_score mean: {is_mean.compute()}, sigma: {is_sigma.compute()}')

87916544/87910968 [==============================] - 0s 0us/step
calculating the inception_score mean ...


300it [2:18:09, 27.07s/it]

calculating the inception_score sigma ...


/usr/local/lib/python3.6/dist-packages/dask/array/routines.py:780: RuntimeWarning: Degrees of freedom <= 0 for slice
  warnings.warn("Degrees of freedom <= 0 for slice", RuntimeWarning)
0it [00:00, ?it/s]/usr/local/lib/python3.6/dist-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))
1it [00:27, 27.06s/it]/usr/local/lib/python3.6/dist-packages/dask/array/core.py:3967: RuntimeWarning: invalid value encountered in double_scalars
  result = function(*args, **kwargs)
2it [00:53, 26.97s/it]/usr/local/lib/python3.6/dist-packages/dask/array/core.py:3967: RuntimeWarning: invalid value encountered in double_scalars
  result = function(*args, **kwargs)
3it [01:20, 26.96s/it]/usr/local/lib/python3.6/dist-packages/dask/array/core.py:3967: RuntimeWarning: invalid value encountered in double_scalars
  result = function(*args, **kwargs)
4it [01:47, 26.86s/it]/usr/local/lib/python3.6/dist-packages/dask/array/core.

inception_score mean: [5.71726667e+157], sigma: inf



## Frechet_inception_distance




In [ ]:
from evaluation.generativity_metrics.inception_metrics import frechet_inception_distance

In [ ]:
fis_score = frechet_inception_distance(ae, train_generator.as_numpy_iterator(), 
                                       tolerance_threshold=1e-6, 
                                       max_iteration=300, 
                                       batch_size=32)


In [ ]:
print(f'frechet inception distance: {fis_score.compute()}')

## perceptual_path_length_score

In [ ]:
from evaluation.generativity_metrics.perceptual_path_length import perceptual_path_length_score

In [ ]:
ppl_mean_score = perceptual_path_length_score(ae, train_generator.as_numpy_iterator(), 
                                              tolerance_threshold=1e-6, 
                                              max_iteration=300, 
                                              batch_size=32)
print(f'perceptual path length score: {ppl_mean_score}')

58892288/58889256 [==============================] - 1s 0us/step


300it [04:24,  1.13it/s]

perceptual path length score: 5193.7294921875


## precision score

In [ ]:
from evaluation.generativity_metrics.precision_recall import precision_score

In [ ]:
_precision_score = precision_score(ae,  train_generator.as_numpy_iterator(), 
                                   tolerance_threshold=1e-6, 
                                   max_iteration=300)
print(f'precision score: {_precision_score}')

/content/Generative_Models/evaluation/generativity_metrics/precision_recall.py:84: RuntimeWarning: invalid value encountered in true_divide
  realism_score[begin1:end1] = self.D[nearest_indices[begin1:end1], 0] / np.min(distance_batch[0:end1-begin1, :], axis=1)
                  

Evaluating k-NN precision and recall with 10 samples...
Done evaluation in: 0.00248241s
Evaluating k-NN precision and recall with 10 samples...
Done evaluation in: 0.00180984s
precision score: [0.]


## recall score

In [ ]:
from evaluation.generativity_metrics.precision_recall import recall_score

In [ ]:
_recall_score = recall_score(ae, train_generator.as_numpy_iterator(), tolerance_threshold=1e-6, max_iteration=300)
print(f'recall score: {_recall_score}')

Evaluating k-NN precision and recall with 10 samples...
Done evaluation in: 0.00187397s
Evaluating k-NN precision and recall with 10 samples...
Done evaluation in: 0.00173426s
recall score: [1.]


# Image Generation

## image reconstruction

### Training dataset

In [ ]:
from training.generators.from_images.image_generation_testing import reconstruct_from_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'reconstruct_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, train_generator.as_numpy_iterator(), save_dir)

10it [00:03,  2.60it/s]


In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'reconstruct_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, val_generator.as_numpy_iterator(), save_dir)

10it [00:04,  2.29it/s]


## with Randomness

In [ ]:
from training.generators.from_images.image_generation_testing import generate_images_like_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'generate_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, train_generator.as_numpy_iterator(), save_dir)

10it [00:04,  2.23it/s]


In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'generate_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, val_generator.as_numpy_iterator(), save_dir)

10it [00:04,  2.14it/s]


### Complete Randomness

In [ ]:
from training.generators.from_images.image_generation_testing import generate_images_randomly

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'random_synthetic_dir')
create_if_not_exist(save_dir)

generate_images_randomly(ae, save_dir)

32it [00:07,  4.28it/s]


In [ ]:
from training.generators.from_images.image_generation_testing import interpolate_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'interpolate_dir')
create_if_not_exist(save_dir)

interpolate_a_batch(ae, val_generator.as_numpy_iterator(), save_dir)

10it [00:00, 131.89it/s]
10it [00:00, 143.70it/s]
0it [00:00, ?it/s]

10it [00:00, 132.60it/s]
10it [00:00, 138.24it/s]
100%|██████████| 4/4 [00:00<00:00, 11.49it/s]
10it [00:00, 129.97it/s]
